<a href="https://colab.research.google.com/github/pawelrat009/ML2025-1/blob/main/PUM11_zadanie_system_rekomendacyjny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.23.5

In [7]:
import pandas as pd

In [8]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
movie_titles = dict(zip(movies['movieId'], movies['title']))

In [9]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2461549 sha256=e8bf281b323f157f565f02fea0fadd37dda38f2e18363ae2d6ebdfb04652955a
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [11]:
# importing relevant libraries
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import Reader

In [13]:
# Get minimum and maximum rating from the dataset
min_rating = ratings.rating.min()
max_rating = ratings.rating.max()
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [14]:
# algorytm SVD
algo = SVD()

# walidacja krzyżowa
cross_validate(algo, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8764  0.8744  0.8630  0.8809  0.8743  0.8738  0.0059  
MAE (testset)     0.6731  0.6710  0.6650  0.6788  0.6703  0.6716  0.0044  
Fit time          1.99    1.50    1.51    1.51    1.49    1.60    0.20    
Test time         0.23    0.24    0.13    0.11    0.26    0.19    0.06    


{'test_rmse': array([0.87642967, 0.87444989, 0.86304965, 0.88093787, 0.87434727]),
 'test_mae': array([0.67308707, 0.67102778, 0.66503172, 0.67875923, 0.67026011]),
 'fit_time': (1.9889781475067139,
  1.5017621517181396,
  1.5113410949707031,
  1.5068840980529785,
  1.4864964485168457),
 'test_time': (0.23185372352600098,
  0.24416089057922363,
  0.1261894702911377,
  0.11214423179626465,
  0.25968027114868164)}

In [21]:
# Trenowanie modelu
trainset = data.build_full_trainset()
algo.fit(trainset)

# rekomendacje dla użytkownika o ID=600
user_id = 600
movies_watched_by_user = ratings[ratings['userId'] == user_id]['movieId'].tolist()
movies_not_watched = movies[~movies['movieId'].isin(movies_watched_by_user)]['movieId'].tolist()

recommendations = []
for movie_id in movies_not_watched:
    prediction = algo.predict(user_id, movie_id)
    recommendations.append((movie_id, prediction.est))

# sortowanie wyników
recommendations.sort(key=lambda x: x[1], reverse=True)

# wyświetlenie wyników
print("10 rekomendacji filmów dla użytkownika o id=600:")
print(f"Rekomendacja (ocena predykcyjna) --- Tytuł filmu (rok produkcji)")
for movie_id, rating in recommendations[:10]:
    print(f"{round(rating, 4)} --- {movie_titles[movie_id]}")

10 rekomendacji filmów dla użytkownika o id=600:
Rekomendacja (ocena predykcyjna) --- Tytuł filmu (rok produkcji)
4.2382 --- Departed, The (2006)
4.1884 --- Godfather: Part II, The (1974)
4.1756 --- Usual Suspects, The (1995)
4.1313 --- Donnie Darko (2001)
4.1124 --- Outlaw Josey Wales, The (1976)
4.0951 --- Paths of Glory (1957)
4.0697 --- Wallace & Gromit: The Best of Aardman Animation (1996)
4.0234 --- Grand Day Out with Wallace and Gromit, A (1989)
3.9987 --- All the President's Men (1976)
3.9859 --- Breakfast Club, The (1985)
